In [56]:

import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer, pipeline
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from afinn import Afinn
from tqdm import tqdm
import unidecode
from time import sleep
logging.basicConfig(level=logging.INFO)# OPTIONAL

In [57]:
print(f"PyTorch version: {torch.__version__}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 1.13.1
Using device: mps


https://github.com/valeriobasile/hurtlex


In [58]:
def strip_accent(accented_string):
    return unidecode.unidecode(str(accented_string))


def HurtlexSetup():
    language = 'en'
    #assert language in ['it', 'fr', 'es', 'pt', 'ro', 'en']
    hurtlex = pd.read_csv(f"https://raw.githubusercontent.com/MilaNLProc/hurtlex/master/lexica/{language.upper()}/1.2/hurtlex_{language.upper()}.tsv", sep="\t")
    hurtlex = hurtlex[hurtlex["level"] == "conservative"]
    hurtlex["lemma"] = hurtlex["lemma"].apply(strip_accent)
    #categories = set(self.hurtlex["category"].unique())
    #words = set(self.hurtlex["lemma"].unique())
    return hurtlex

def get_hurtlex_category(hurtlex, lemma):
    try:
        return hurtlex[hurtlex["lemma"] == strip_accent(lemma)]["category"].values[0]
    except:
        return ''

In [59]:
def SentimentAnalysis(afn, word):
    return afn.score(word)

In [60]:
def _sum(arr):
    sum = 0
    for i in arr:
        sum = sum + i
    return(sum)
    
def Classifier_SD():
    model = AutoModelForSequenceClassification.from_pretrained('NLP-LTU/bertweet-large-sexism-detector')
    tokenizer = AutoTokenizer.from_pretrained('NLP-LTU/bertweet-large-sexism-detector') 
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    return classifier

def SexismDetectorEvaluator(classifier ,sentence):
    prediction=classifier(sentence)
    #print(sentence)
    #print(prediction[0]['label'])
    if(prediction[0]['label'] == "sexist"):
        return 1, prediction[0]['score']
    else: 
        return 0, prediction[0]['score']


In [61]:
class OpenPrediction():
    def __init__(self, template_file, target_file, model_name, numAtt):
        self.template_file = template_file
        self.target_file = target_file
        self.numAtt = numAtt
        self.data = []
        self.model_name = model_name
        self.model = BertForMaskedLM.from_pretrained(model_name)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model.eval()
        self.createTemplate()

    def createTemplate(self):
        mask = "[MASK]"
        target = '<target>'
        s='______'
        gender = {'female':1, 'male':2} 
        dataList =[]
        #For each sentence in the template
        for index,row in tqdm(self.template_file.iterrows(), total=self.template_file.shape[0], desc='Creating template', unit='sentences'):
            sentence = row.loc['template']
            #For each target coherent with the template
            for ind, r in self.target_file.iterrows():
                if r.loc['target'] == row.loc['target']:
                        adjectiveList = []
                        #For both gender
                        for t in gender.keys():
                            _sentence = re.sub(target, r.loc[t], sentence)  
                            _sentence = re.sub(s, mask, _sentence) 
                            adjectiveList = self.predict_masked_sent(_sentence) 
                            sentencesNew = []
                            #detector_scores = []
                            for a in adjectiveList:
                                #print(f"{a}")
                                #print(f"{_sentence}")
                                comp_sentence = re.sub('\[MASK\]', a, _sentence)
                                #print(f"{comp_sentence}")
                                sentencesNew.append(comp_sentence)                                
                            data=[
                                sentence, #template
                                r.loc[t], #subject
                                adjectiveList, #word list
                                sentencesNew, #sentence list
                                #detector_scores #scores 
                            ]
                            dataList.append(data)
                            #print(dataList)
        data_df = pd.DataFrame(dataList, columns=["template", "target", "attributes", "sentences"])
        self.data = data_df
        #print(data_df)
        #return dataList

    #Given a number n and a sentence containing a [MASK], it generates the top n words that fits the MASK 
    def predict_masked_sent(self, text):
        # Tokenize input
        text = "[CLS] %s [SEP]"%text
        #print(f"text: {text}")
        tokenized_text = self.tokenizer.tokenize(text)
        masked_index = tokenized_text.index("[MASK]")
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]

        probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
        top_k_weights, top_k_indices = torch.topk(probs, self.numAtt, sorted=True)

        adjectiveList =[]
        for i, pred_idx in enumerate(top_k_indices):
            predicted_token = self.tokenizer.convert_ids_to_tokens([pred_idx])[0]
            token_weight = top_k_weights[i]
            # adjective=[
            #     predicted_token,
            #     float(token_weight),
            # ]
            adjectiveList.append(predicted_token)
            #print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight)*1000)
        return adjectiveList
    
    def evaluate_predictions(self):
        sexist_detector_label = []
        sexism_detector_count = []
        hurtlex = []
        sent_analysis = []
        afn = Afinn()
        classifier = Classifier_SD()
        hurtlex_evaluator = HurtlexSetup()
        for index,row in tqdm(self.data.iterrows(), total=self.data.shape[0], desc='Evaluating sentences', unit='sentences'):
            sleep(0.0001)
            adj_lbl = []
            sent_score = []
            for sent in row['sentences']:
                label, score = SexismDetectorEvaluator(classifier, sent)
                adj_lbl.append(label)
                sent_score.append(score)
            sexism_detector_count.append(_sum(adj_lbl))
            sexist_detector_label.append(adj_lbl)
            hurtlex_tmp = []
            sent_analysis_tmp = []
            for wrd in row['attributes']:
                    hurtlex_tmp.append(get_hurtlex_category(hurtlex_evaluator, wrd))
                    sent_analysis_tmp.append(SentimentAnalysis(afn, wrd))
            hurtlex.append(hurtlex_tmp)
            sent_analysis.append(sent_analysis_tmp)
        self.data.loc[:,'sexism detector'] = sexist_detector_label
        self.data.loc[:,'sexism detector count'] = sexism_detector_count
        self.data.loc[:,'hurtlex'] = hurtlex
        self.data.loc[:,'sentiment analysis'] = sent_analysis
        self.data.to_csv(f"../src/result/sexisminsocietal_{self.model_name}_results.csv", sep=";")

In [62]:
templates = pd.read_csv('dataset/template_societal.csv', sep=";")
target_file = pd.read_csv('dataset/template_subjects.csv', sep=";")
numAtt = 10
model_name = 'bert-base-uncased'
evaluator = OpenPrediction(templates.copy(), target_file.copy(), model_name, numAtt)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating template: 100%|██████████| 267/267 [01:23<00:00,  3.20sentences/s]


In [63]:
evaluator.evaluate_predictions()

Evaluating sentences: 100%|██████████| 1834/1834 [1:26:55<00:00,  2.84s/sentences]    
